In [1]:
import pandas as pd
import os

In [2]:
# Read the problems file
problems = pd.read_json('code4bench_no_loops.json')
loop_problems = pd.read_json('code4bench_loops.json')


In [4]:
# For each of the problems make a folder with the index
for i, problem in problems.iterrows():
    # define the range of problems to extract here
    if i > 2000 and i < 2200:
        # Get the context of the problem
        context = problem['context']
        
        # Get the path, start with the current directory and add the folder name
        path = os.path.join(os.getcwd(), "..", "no_loop_problems", f"{i}")

        # Create the folder if it does not exist
        if not os.path.exists(path):
            os.makedirs(path)
        
        # Write the context to a file
        with open(os.path.join(path, 'context.txt'), 'w', encoding="utf-8") as f:
            f.write(context)

In [13]:
index = 68
print(problems.iloc[index])
print(problems.iloc[index]['sourceCode'])
problems.iloc[index]['sourceCode'].replace("\r", " ").replace("\n", " ").replace("\t", " ")

id                                                         2490
sourceCode    #include<stdio.h>\r\nint main()\r\n{\r\nlong l...
fullname                                                    84A
name                                                          A
countline                                                     7
context       A. Toy Army time limit per test 2 seconds memo...
index                                                        68
Name: 2073, dtype: object
#include<stdio.h>
int main()
{
long long int n;
scanf("%I64d",&n);
printf("%I64d",(3*n)/2);
return(0);}


'#include<stdio.h>  int main()  {  long long int n;  scanf("%I64d",&n);  printf("%I64d",(3*n)/2);  return(0);}'

In [5]:
# Select a random problem from the loop problems
problem = loop_problems.sample()

# Get the index of the problem
i = problem.index[0]

# If the problem does not have a loop then generate a new one. Do this in a loop
while not problem['sourceCode'].str.contains('for|while|do').any():
    problem = loop_problems.sample()

# Get the context of the problem
context = problem['context'].values[0]

# Get the path, start with the current directory and add the folder name
path = os.path.join(os.getcwd(), "..", "loop_problems", f"{i}")

# Create the folder if it does not exist
if not os.path.exists(path):
    os.makedirs(path)

# Write the context to a file
with open(os.path.join(path, 'context.txt'), 'w', encoding="utf-8") as f:
    f.write(context)
    
# Also write the context commented to the source code
with open(os.path.join(path, 'solution.c'), 'w', encoding="utf-8") as f:
    f.write(f"/* {context} */\n\n")
    f.write(problem['sourceCode'].values[0].replace("\r", " ").replace("\t", " "))
    